In [2]:
from nautilus_trader.model.enums import BookAction
from nautilus_trader.model.enums import BookType
from nautilus_trader.model.orderbook.data import OrderBookDeltas
from nautilus_trader.model.orderbook.data import Order
from nautilus_trader.model.orderbook.data import OrderBookDelta
from nautilus_trader.model.orderbook.book import OrderBook
# from nautilus_trader.model.orderbook.book import L2OrderBook
from nautilus_trader.model.identifiers import InstrumentId, Symbol, Venue
from nautilus_trader.backtest.data.providers import TestInstrumentProvider
from collections import defaultdict
from nautilus_trader.examples.strategies.market_maker import MarketMaker
from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.backtest.engine import BacktestEngineConfig
from nautilus_trader.model.identifiers import Venue
from nautilus_trader.model.objects import Money
from nautilus_trader.model.enums import OMSType
from nautilus_trader.model.enums import AccountType
from nautilus_trader.model.currencies import USD, USDT, BTC
from decimal import Decimal

import pandas as pd

In [3]:
bin_btcusdt_ticks_file = ''
data_file = '../../data/l2_data_extract_discord/BINANCE-BTC-USDT-incremental_l2_book-2022-10-13-00-00-00-2022-10-14-00-00-00.csv.gz'

data_file = '/Users/andrewgoldberg/Projects/hummingbot-backtest/data/l2_data_extract_discord/BINANCE-BTC-USDT-incremental_l2_book-2022-10-13-00-00-00-2022-10-14-00-00-00.csv.gz'
data_df = pd.read_csv(data_file, nrows=10000, compression='gzip', error_bad_lines=False)

/var/folders/c7/8ppmyxcn6znc4ql4nfmfksy80000gn/T/ipykernel_59030/400293680.py:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data_df = pd.read_csv(data_file, nrows=10000, compression='gzip', error_bad_lines=False)


In [4]:
data_df

,Unnamed: 0,exchange,symbol,timestamp,local_timestamp,is_snapshot,side,price,amount
0,0,BINANCE,BTC-USDT,1665619199929000,1665619200040000,False,bid,19155.40,0.00000
1,1,BINANCE,BTC-USDT,1665619199929000,1665619200040000,False,bid,19155.09,0.02510
2,2,BINANCE,BTC-USDT,1665619199929000,1665619200040000,False,bid,19154.49,0.00500
3,3,BINANCE,BTC-USDT,1665619199929000,1665619200040000,False,bid,19154.48,0.00000
4,4,BINANCE,BTC-USDT,1665619199929000,1665619200040000,False,bid,19154.47,0.00000
...,...,...,...,...,...,...,...,...,...
9995,9995,BINANCE,BTC-USDT,1665619220339000,1665619220453000,False,ask,19166.33,1.01106
9996,9996,BINANCE,BTC-USDT,1665619220339000,1665619220453000,False,ask,19168.13,0.00000
9997,9997,BINANCE,BTC-USDT,1665619220339000,1665619220453000,False,ask,19168.74,0.00522
9998,9998,BINANCE,BTC-USDT,1665619220339000,1665619220453000,False,ask,19168.86,0.00000


In [5]:

side_to_code = {
    'bid': 1,
    'ask': 2
}

In [6]:
data_df.iloc[0, :]

Unnamed: 0                        0
exchange                    BINANCE
symbol                     BTC-USDT
timestamp          1665619199929000
local_timestamp    1665619200040000
is_snapshot                   False
side                            bid
price                       19155.4
amount                          0.0
Name: 0, dtype: object

In [7]:
BIN_BTCUSDT = TestInstrumentProvider.btcusdt_binance()

In [8]:
alldeltas = []
for ts in data_df.timestamp.unique():
    deltas = []
    for idx, row in data_df[data_df.timestamp==ts].iterrows():
        delta_ = OrderBookDelta(
            instrument_id=BIN_BTCUSDT.id,
            book_type=BookType.L2_MBP,
            action=BookAction.DELETE if row.amount == 0 else BookAction.UPDATE,
            order=Order(
                price=row['price'],
                size=row['amount'],
                side=side_to_code[row['side']]
            ),
            ts_event=ts*1000,
            ts_init=ts*1000
        )
        deltas.append(delta_)
    alldeltas.append(
        OrderBookDeltas(
            book_type=BookType.L2_MBP,
            instrument_id=BIN_BTCUSDT.id,
            deltas=deltas,
            ts_event=ts*1000,
            ts_init=ts*1000,
        )
        )


In [9]:
# from nautilus_trader.persistence.catalog import ParquetDataCatalog

# catalog = ParquetDataCatalog("./")
# catalog.instruments()

# write_objects(catalog, alldeltas)
# write_objects(catalog, [BIN_BTCUSDT])

In [10]:
# catalog.order_book_deltas()

In [11]:
alldeltas[0]

OrderBookDeltas('BTCUSDT.BINANCE', book_type=L2_MBP, [OrderBookDelta('BTCUSDT.BINANCE', book_type=L2_MBP, action=DELETE, order=Order(19155.4, 0.0, BUY, 7989e4d3-0007-49a9-9233-1597d3c2d7a8), update_id=0, ts_event=1665619199929000000, ts_init=1665619199929000000), OrderBookDelta('BTCUSDT.BINANCE', book_type=L2_MBP, action=UPDATE, order=Order(19155.09, 0.0251, BUY, 90c1265b-f4a8-4973-aee2-e410c308ac39), update_id=0, ts_event=1665619199929000000, ts_init=1665619199929000000), OrderBookDelta('BTCUSDT.BINANCE', book_type=L2_MBP, action=UPDATE, order=Order(19154.49, 0.005, BUY, 0e3eab78-0d31-45ab-ac00-600fc6c9034e), update_id=0, ts_event=1665619199929000000, ts_init=1665619199929000000), OrderBookDelta('BTCUSDT.BINANCE', book_type=L2_MBP, action=DELETE, order=Order(19154.48, 0.0, BUY, ed011223-0821-466a-824f-90931415e029), update_id=0, ts_event=1665619199929000000, ts_init=1665619199929000000), OrderBookDelta('BTCUSDT.BINANCE', book_type=L2_MBP, action=DELETE, order=Order(19154.47, 0.0, BUY,

In [12]:
BIN_BTCUSDT

CurrencyPair(id=BTCUSDT.BINANCE, native_symbol=BTCUSDT, asset_class=CRYPTO, asset_type=SPOT, quote_currency=USDT, is_inverse=False, price_precision=2, price_increment=0.01, size_precision=6, size_increment=0.000001, multiplier=1, lot_size=None, margin_init=0, margin_maint=0, maker_fee=0.001, taker_fee=0.001, info=None)

In [13]:
print('start')
# Configure backtest engine
config = BacktestEngineConfig(
    trader_id="BACKTESTER-001",
)
print('config')
# Build the backtest engine
engine = BacktestEngine(config=config)
print('set config')
# Add a trading venue (multiple venues possible)
engine.add_venue(
    venue=BIN_BTCUSDT.venue,
    oms_type=OMSType.NETTING,
    account_type=AccountType.CASH,
    book_type=2,
    base_currency=BTC,  # Standard single-currency account
    starting_balances=[Money(10_000_000, BTC)],  # Single-currency or multi-currency accounts
)
print('add venue')
# Add instruments
engine.add_instrument(BIN_BTCUSDT)
print('add instrument')
# Add data
engine.add_data(alldeltas)
print('add deltas data')
strategy = MarketMaker(
    instrument_id = BIN_BTCUSDT.id,
    trade_size = Decimal(1),
    max_size = Decimal(400_000_000),
)
print('set marketmaker strateg')
# strategy = VolatilityMarketMaker(config=config)
engine.add_strategy(strategy=strategy)
print('add strategy')
# input("Press Enter to continue...")  # noqa (always Python 3)

# Run the engine (from start to end of data)
engine.run_streaming()
print('run streaming')
# Optionally view reports
with pd.option_context(
    "display.max_rows",
    100,
    "display.max_columns",
    None,
    "display.width",
    300,
):
    print(engine.trader.generate_account_report(BIN_BTCUSDT))
    print(engine.trader.generate_order_fills_report())
    print(engine.trader.generate_positions_report())

# For repeated backtest runs make sure to reset the engine
engine.reset()

# Good practice to dispose of the object when done
engine.dispose()